In [80]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.cross_validation import KFold

import revrand.basis_functions as bf
from revrand.metrics import smse, msll
from revrand.regression import learn, predict
from revrand.btypes import Parameter, Positive

import revrand.legacygp as gp
import revrand.legacygp.kernels as kern


In [100]:
# Load the data
boston = load_boston()
X = boston.data
y = boston.target - boston.target.mean()

N, D = X.shape
lenscale = 1000.
lenARD = lenscale * np.ones(D)


In [101]:
# Construct basis functions
base = bf.RandomMatern32(Xdim=D, nbases=200, lenscale_init=Parameter(lenARD, Positive())) #\
    #+bf.RandomRBF(Xdim=D, nbases=50, lenscale_init=Parameter(lenARD, Positive()))
    

In [102]:
# Construct a GP kernel to compare to
def kdef(h, k):
    return (h(1e-5, 10., 0.5) * k(kern.matern3on2, [h(1e-5, 1e5, l) for l in lenARD]) +
            k(kern.lognoise, h(-4, 1, 0)))


In [103]:
# Cross val
folds = 5
av_smse, av_msll = 0., 0.
av_smse_gp, av_msll_gp = 0., 0

for i, (tr_ind, ts_ind) in enumerate(KFold(len(y), n_folds=folds, shuffle=True)):
    
    # Training
    params = learn(X[tr_ind], y[tr_ind], base)
     
    #hyper_params = gp.learn(X[tr_ind], y[tr_ind], kdef, verbose=False, ftol=1e-15)
    #regressor = gp.condition(X[tr_ind], y[tr_ind], kdef, hyper_params)
    
    # Prediction
    Ey, Vf, Vy = predict(X[ts_ind], base, *params)
    
    #query = gp.query(regressor, X[ts_ind])
    #Ey_gp = gp.mean(query)
    #Vy_gp = gp.variance(query, noise=True)

    
    # Validation
    f_smse, f_msll = smse(y[ts_ind], Ey), msll(y[ts_ind], Ey, Vy, y[tr_ind])
    #f_smse_gp, f_msll_gp = smse(y[ts_ind], Ey_gp), msll(y[ts_ind], Ey_gp, Vy_gp, y[tr_ind])

    av_smse += f_smse
    av_msll += f_msll
    #av_smse_gp += f_smse_gp
    #av_msll_gp += f_msll_gp
    
    print("Fold: {},\n\trevrand: SMSE = {}, MSLL = {}"
          #"\n\tGP: SMSE = {}, MSLL = {}"
          .format(i, f_smse, f_msll))#, f_smse_gp, f_smse_gp))
    
av_smse /= folds
av_msll /= folds
#av_smse_gp /= folds
#av_msll_gp /= folds

Fold: 0,
	revrand: SMSE = 0.11515026692844521, MSLL = -1.136777829359582
Fold: 1,
	revrand: SMSE = 0.21883184976540862, MSLL = -0.7547862845380295
Fold: 2,
	revrand: SMSE = 0.14010044301498245, MSLL = -1.138129200617719
Fold: 3,
	revrand: SMSE = 0.4171394023359003, MSLL = -0.17629713462449503
Fold: 4,
	revrand: SMSE = 0.0761734422806629, MSLL = -1.3134234137596041


In [104]:
# Print results
print("Final:\n\trevrand: SMSE = {}, MSLL = {}"
      #"\n\tGP: SMSE = {}, MSLL = {}"
      .format(av_smse, av_msll))#, av_smse_gp, av_msll_gp))


Final:
	revrand: SMSE = 0.1934790808650799, MSLL = -0.903882772579886
